<a href="https://colab.research.google.com/github/mzohaibnasir/chromadb_basicRAG/blob/main/basicRAG_chromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
PINECONE_API_KEY = userdata.get("PINECONE_API_KEY")


os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY


# OPENAIAPIKEY,PINECONEAPIKEY

# Chroma Vector DB

pinecone is designed to handle real-time search and similarity matching at scale.. automatic indexing is available but its costly and limited querying functionality

Chromadb is not cloud based but it is smilar to pinecone(designed for vector storage and handle retreval). opensource. extensive querying but deployment complexity and might not be great as pinecone in realtime.


In [ ]:
! pip install langchain
! pip install openai
! pip install tiktoken
! pip install chromadb
!pip install unstructured
# ! pip install pypdf
# ! pip install -U langchain_chroma
# ! pip install -U langchain_openai

In [4]:
! pip show chromadb

Name: chromadb
Version: 0.4.24
Summary: Chroma.
Home-page: 
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pulsar-client, pydantic, pypika, PyYAML, requests, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: 


In [8]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.document_loaders import DirectoryLoader, TextLoader

In [6]:
! wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [7]:
! unzip -q new_articles.zip -d new_articles

In [12]:
loader = DirectoryLoader("/content/new_articles",
                glob = "./*.txt")
document = loader.load()

In [ ]:
len(document), print(document[0].page_content)

In [ ]:
# creating chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200

)

text = text_splitter.split_documents(
    document
)


In [23]:
len(text), text[0]

(233,
 Document(page_content='It’s that time of week again, folks — Week in Review (WiR) time. For those new to the scene, WiR is TechCrunch’s regular newsletter that recaps the biggest tech stories over the past few days. There’s no better digest for the person on the go, we’d argue — but of course, we’re a little biased.\n\nBefore we get into the meat of the thing, a quick reminder that TC City Spotlight: Atlanta is fast approaching. On June 7, TechCrunch is headed to Atlanta, where we’ll host a pitch competition, a talk on the economics of equality, a panel discussion on investing in the Atlanta ecosystem and more.\n\nElsewhere, there’s a TechCrunch Live event with Persona and Index Ventures on May 10, which will touch on how Persona keeps pace with new threats and how Index made a prescient move to spot and back Persona early on. And we have Disrupt in San Francisco from September 19–21 — our annual conference is jam-packed with expert-led sessions and interviews with movers and sh

## creating DB

In [26]:
from langchain import embeddings

In [27]:
persist_directory = 'db'  # local storage directory

In [28]:

embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(
    documents = text,
    embedding=embedding,
    persist_directory = persist_directory
)



### in pinecone we were able to see embeddnigs but in chromedb all embeddings are in binary files(.bin). Chromadb is not a server but a  wrapper and in backend its using sqlite3.

In [30]:
# persist db to disk
vectordb.persist()

In [3]:
vectordb=None

In [31]:
# load persisted database from diks and use it as normal fashion
vectordb = Chroma(
    persist_directory = persist_directory,
    embedding_function = embedding
)
vectordb # so I'm able to persist my data in my local disk

## Make retriever

In [32]:
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7ab2b7a37cd0>)

In [36]:
docs = retriever.get_relevant_documents(
    "tell me about money Micrososft raised?"
)
docs

[Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”', metadata={'source': '/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}),
 Document(page_content='Partners of 3one4 Capital, a v

In [ ]:
"""

Data-> embedding -> chromadb


Retriever makes a query.request goes to db and I get response based on similarity search.
"""

In [37]:
len(docs)

4

In [38]:
retriever = vectordb.as_retriever(
    search_kwargs={"k":2}
)
retriever.search_kwargs

{'k': 2}

In [39]:
docs2 = retriever.get_relevant_documents("how much money did Microsoft raised?")
len(docs2)

2

In [40]:
docs2

[Document(page_content='April 28, 2023\n\nVC firms including Sequoia Capital, Andreessen Horowitz, Thrive and K2 Global are picking up new shares, according to documents seen by TechCrunch. A source tells us Founders Fund is also investing. Altogether the VCs have put in just over $300 million at a valuation of $27 billion to $29 billion. This is separate to a big investment from Microsoft announced earlier this year, a person familiar with the development told TechCrunch, which closed in January. The size of Microsoft’s investment is believed to be around $10 billion, a figure we confirmed with our source.\n\nApril 25, 2023\n\nCalled ChatGPT Business, OpenAI describes the forthcoming offering as “for professionals who need more control over their data as well as enterprises seeking to manage their end users.”', metadata={'source': '/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}),
 Document(page_content='Partners of 3one4 Capital, a v

## make it more realistic
###Make a chain

In [41]:
from langchain.chains import RetrievalQA

In [43]:
llm = OpenAI()
# llm

In [47]:
# creating a chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm =llm,
    chain_type = "stuff",
    retriever=retriever,
    return_source_documents = True
)
# qa_chain

In [51]:
# cite sources
def process_llm_response(llm_response):
  print(llm_response['result'])
  print('\n\nSources:')
  for source in llm_response["source_documents"]:
    print(source.metadata["source"])

In [52]:
query = "how much money did Microsoft raised?"
llm_response=qa_chain(query)
process_llm_response(llm_response)

 $10 billion


Sources:
/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt
/content/new_articles/05-07-3one4-capital-driven-by-contrarian-bets-raises-200-million-new-fund.txt


In [60]:
llm_response['source_documents'][0].metadata

{'source': '/content/new_articles/05-03-chatgpt-everything-you-need-to-know-about-the-ai-powered-chatbot.txt'}

# THIS is the basic RAG we created!

Answer is not coming from model but from database(embvbedding. LLM is here just for refinement just like function calling.

In [61]:
# cleaning database

!zip -r db.zip ./db

# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()


# delete the directory
!rm -rf db/

  adding: db/ (stored 0%)
  adding: db/chroma.sqlite3 (deflated 41%)
  adding: db/67436a60-3a65-4267-b09c-0cc801a434e9/ (stored 0%)
  adding: db/67436a60-3a65-4267-b09c-0cc801a434e9/header.bin (deflated 61%)
  adding: db/67436a60-3a65-4267-b09c-0cc801a434e9/length.bin (deflated 52%)
  adding: db/67436a60-3a65-4267-b09c-0cc801a434e9/data_level0.bin (deflated 100%)
  adding: db/67436a60-3a65-4267-b09c-0cc801a434e9/link_lists.bin (stored 0%)
